In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Computer_vision
!pwd
!nvidia-smi

/content/drive/MyDrive/Computer_vision
/content/drive/MyDrive/Computer_vision
Wed Jun  4 14:09:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                   

In [ ]:
# Pip install method (recommended)
%cd /content/drive/MyDrive/Computer_vision
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
%cd /content/drive/MyDrive/Computer_vision

/content/drive/MyDrive/Computer_vision


In [ ]:
!yolo task=segment mode=train model=yolov8n-seg.pt data=OTU2D_8_layers_merged_preprocessed_no_augmented_yolo/data.yaml batch=32 epochs=50 imgsz=640 augment=False


Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=OTU2D_8_layers_merged_preprocessed_no_augmented_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective

In [ ]:
!pip install torch torchvision torchmetrics opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 22.6 MB/s eta 0:00:00


In [ ]:
import os
import yaml
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
from ultralytics import YOLO
from torchmetrics.classification import BinaryF1Score, BinaryJaccardIndex

yaml_path = 'OTU2D_8_layers_merged_preprocessed_no_augmented_yolo/data.yaml'
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

dataset_root = Path(data['path'])
test_img_dir = dataset_root / data['test']
test_label_dir = dataset_root / 'test' / 'labels'

model = YOLO('runs/segment/train3/weights/best.pt')

# === Precision, Recall, mAP ===
metrics = model.val(
    data=yaml_path,
    split='test',
    iou=0.5,
    conf=0.5,
    task='segment',
    save=False,
    verbose=False
)

precision = metrics.seg.p[0]
recall = metrics.seg.r[0]
map50 = metrics.seg.map50
map5095 = metrics.seg.map

print(f"\n🎯 Precision: {precision:.4f}")
print(f"🎯 Recall:    {recall:.4f}")
print(f"📊 mAP@0.5:   {map50:.4f}")
print(f"📊 mAP@0.5:0.95: {map5095:.4f}")

# === Dice, IoU ===
dice_metric = BinaryF1Score().to('cuda')
iou_metric = BinaryJaccardIndex().to('cuda')
dice_scores, iou_scores = [], []

def render_mask(label_file, img_shape):
    H, W = img_shape
    mask = np.zeros((H, W), dtype=np.uint8)
    if not os.path.exists(label_file):
        return mask

    with open(label_file, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split()))
            if len(parts) < 6:
                continue
            coords = parts[1:]
            points = np.array(coords).reshape(-1, 2) * [W, H]
            points = points.astype(np.int32)
            cv2.fillPoly(mask, [points], color=1)
    return mask

for img_path in tqdm(sorted(test_img_dir.glob('*.jpg'))):
    img = cv2.imread(str(img_path))
    H, W = img.shape[:2]
    label_path = test_label_dir / (img_path.stem + '.txt')


    results = model.predict(source=img, task='segment', verbose=False)
    pred_masks = results[0].masks.data if results[0].masks else None

    if pred_masks is None or len(pred_masks) == 0:
        pred_mask_bin = torch.zeros((1, H, W), dtype=torch.float32).to('cuda')
    else:
        pred_mask_bin = torch.any(pred_masks, dim=0).float().unsqueeze(0).to('cuda')

    gt_mask_np = render_mask(str(label_path), (H, W))
    gt_mask_bin = torch.tensor(gt_mask_np, dtype=torch.float32).unsqueeze(0).to('cuda')


    dice = dice_metric(pred_mask_bin, gt_mask_bin)
    iou = iou_metric(pred_mask_bin, gt_mask_bin)

    dice_scores.append(dice.item())
    iou_scores.append(iou.item())

print(f"\n📈 Mean Dice (F1): {np.mean(dice_scores):.4f}")
print(f"📈 Mean mIoU:      {np.mean(iou_scores):.4f}")


Ultralytics 8.3.148 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs
val: Fast image access ✅ (ping: 1.8±3.4 ms, read: 12.4±15.6 MB/s, size: 70.3 KB)


val: Scanning /content/drive/MyDrive/Computer_vision/OTU2D_8_layers_merged_preprocessed_no_augmented_yolo/test/labels.cache... 237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 237/237 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:06<00:00,  2.48it/s]


                   all        237        240      0.886      0.846      0.893      0.712      0.886      0.846      0.893      0.706
Speed: 1.5ms preprocess, 7.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/segment/val15

🎯 Precision: 0.8865
🎯 Recall:    0.8458
📊 mAP@0.5:   0.8925
📊 mAP@0.5:0.95: 0.7057


100%|██████████| 237/237 [00:06<00:00, 37.76it/s]


📈 Mean Dice (F1): 0.8390
📈 Mean mIoU:      0.7702
